In [22]:
import pandas as pd
import zipfile
import sqlite3
import os
# non IMDB 
bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
tmbd = pd.read_csv('zippedData/tmdb.movies.csv.gz')
tn_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
rt_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz',sep='\t')
rt_review = pd.read_csv('zippedData/rt.reviews.tsv.gz',sep='\t',encoding='latin-1')

In [23]:
extraction_path = 'zippedData/' 
zip_file_path = 'zippedData/im.db.zip'
with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# The .db file should now be in the extraction_path directory
# Assuming the .db file has the same name as the .zip file
db_file_name = os.path.basename(zip_file_path).replace('.zip', '')
db_file_path = os.path.join(extraction_path, db_file_name)

# Connect to the SQLite database
conn = sqlite3.connect(db_file_path)

# Perform database operations
cursor = conn.cursor()

# For example, to print the list of tables
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cursor.fetchall())

query_movie_basics = ("SELECT * FROM movie_basics")
query_directors = ("SELECT * FROM directors")
query_known_for = ("SELECT * FROM known_for")
query_movie_akas = ("SELECT * FROM movie_akas")
query_movie_ratings = ("SELECT * FROM movie_ratings")
query_persons = ("SELECT * FROM persons")
query_principals = ("SELECT * FROM principals")
query_writers = ("SELECT * FROM writers")

big_query = (
"""
SELECT 
movie.movie_id,
movie.start_year,
movie.primary_title,
movie.original_title,
movie.genres,
movie.runtime_minutes,
MAX(IIF(principals.category='director',persons.primary_name,NULL)) as director_name,
MAX(IIF(principals.category='writer',persons.primary_name,NULL)) as writer_name,
GROUP_CONCAT(IIF(principals.category='actor',persons.primary_name,NULL),', ') as actors
FROM movie_basics as movie
LEFT JOIN principals 
    ON movie.movie_id = principals.movie_id 
LEFT JOIN persons 
    ON persons.person_id = principals.person_id
GROUP BY 1
"""
)



movie_basics_df = pd.read_sql_query(query_movie_basics,conn)
directors_df = pd.read_sql_query(query_directors,conn)
known_for_df = pd.read_sql_query(query_known_for,conn)
movie_akas_df = pd.read_sql_query(query_movie_akas,conn)
movie_ratings_df = pd.read_sql_query(query_movie_ratings,conn)
persons_df = pd.read_sql_query(query_persons,conn)
principals_df = pd.read_sql_query(query_principals,conn)
writers_df = pd.read_sql_query(query_writers,conn)

big_df = pd.read_sql_query(big_query,conn)

conn.close()

In [24]:
big_df.head()

,movie_id,start_year,primary_title,original_title,genres,runtime_minutes,director_name,writer_name,actors
0,tt0063540,2013,Sunghursh,Sunghursh,"Action,Crime,Drama",175.0,Harnam Singh Rawail,Mahasweta Devi,"Dilip Kumar, Sanjeev Kumar, Balraj Sahni"
1,tt0066787,2019,One Day Before the Rainy Season,Ashad Ka Ek Din,"Biography,Drama",114.0,Mani Kaul,None,"Arun Khopkar, Om Shivpuri"
2,tt0069049,2018,The Other Side of the Wind,The Other Side of the Wind,Drama,122.0,Orson Welles,None,"Peter Bogdanovich, John Huston"
3,tt0069204,2018,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama",NaN,Hrishikesh Mukherjee,Gulzar,"Vijay Arora, Utpal Dutt, Rabi Ghosh, Asrani"
4,tt0100275,2017,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy",80.0,Valeria Sarmiento,Pía Rey,"Luis Alarcón, Francisco Reyes"


In [25]:
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [26]:
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [27]:
tmbd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [28]:
tn_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [29]:
rt_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [30]:
rt_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [31]:
# need to check for movie name title discrepencies 

In [32]:
rt_review.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [33]:
rt_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [34]:
#could be useful for foreign language films
movie_akas_df.head()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [35]:
directors_df.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [36]:
known_for_df.head()

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534


In [37]:
persons_df.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [38]:
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [39]:
tn_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [40]:
#Use tn_budgets for production cost, domestic gross, foreign gross, release date
#directors_df combined with principles_df for directors
#movie_basics_df has runtime and genre, and title
#persons_df has profession and name (for actors)
#movie_ratings_df has user ratings
#connecting tn_budgets to imdb databases with tn_budgets movie to movie basics origional title or primary title
#need to establish budget floor and release date floor
#not using rotten tomatoes as it has no movie title for some reason

In [41]:
tn_budgets.head()

tn_budgets['year'] = pd.DatetimeIndex(tn_budgets['release_date']).year

tn_budgets[tn_budgets['movie']=='On the Road']

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
2316,17,"Mar 22, 2013",On the Road,"$25,000,000","$720,828","$9,313,302",2013


In [42]:
merged_df = pd.merge(big_df, tn_budgets, left_on=['primary_title','start_year'],right_on=['movie','year'])


gross = pd.to_numeric(merged_df['worldwide_gross'].str.replace('\$', '', regex=True).str.replace(',',''))
production = pd.to_numeric(merged_df['production_budget'].str.replace('\$', '', regex=True).str.replace(',',''))
domestic = pd.to_numeric(merged_df['domestic_gross'].str.replace('\$', '', regex=True).str.replace(',',''))

merged_df['worldwide_gross'] = gross
merged_df['production_budget'] = production
merged_df['domestic_gross'] = domestic
merged_df['Profit'] = gross - production


merged_df['Profit:Production Ratio'] = (merged_df['Profit']/production)

merged_df = merged_df[merged_df['worldwide_gross']!='$0']

merged_df['release_date'] = pd.to_datetime(merged_df['release_date'])

merged_df = merged_df.dropna(subset=['genres'])
merged_df = merged_df.dropna(subset=['runtime_minutes'])

merged_df = merged_df[merged_df['release_date'] >= '2000-01-01']

merged_df = merged_df[merged_df['production_budget'] >= 1000000]

merged_df

,movie_id,start_year,primary_title,original_title,genres,runtime_minutes,director_name,writer_name,actors,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,Profit,Profit:Production Ratio
0,tt0249516,2012,Foodfight!,Foodfight!,"Action,Animation,Comedy",91.0,Lawrence Kasanoff,Sean Catherine Derek,Charlie Sheen,26,2012-12-31,Foodfight!,45000000,0,73706,2012,-44926294,-0.998362
1,tt0359950,2013,The Secret Life of Walter Mitty,The Secret Life of Walter Mitty,"Adventure,Comedy,Drama",114.0,None,Steve Conrad,"Ben Stiller, Jon Daly",37,2013-12-25,The Secret Life of Walter Mitty,91000000,58236838,187861183,2013,96861183,1.064409
2,tt0365907,2014,A Walk Among the Tombstones,A Walk Among the Tombstones,"Action,Crime,Drama",114.0,Scott Frank,Lawrence Block,"Liam Neeson, David Harbour, Dan Stevens, Boyd ...",67,2014-09-19,A Walk Among the Tombstones,28000000,26017685,62108587,2014,34108587,1.218164
3,tt0369610,2015,Jurassic World,Jurassic World,"Action,Adventure,Sci-Fi",124.0,Colin Trevorrow,Rick Jaffa,"Chris Pratt, Ty Simpkins",34,2015-06-12,Jurassic World,215000000,652270625,1648854864,2015,1433854864,6.669092
4,tt0376136,2011,The Rum Diary,The Rum Diary,"Comedy,Drama",119.0,Bruce Robinson,Hunter S. Thompson,"Johnny Depp, Giovanni Ribisi, Aaron Eckhart, M...",16,2011-10-28,The Rum Diary,45000000,13109815,21544732,2011,-23455268,-0.521228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,tt8155288,2019,Happy Death Day 2U,Happy Death Day 2U,"Drama,Horror,Mystery",100.0,Christopher Landon,Scott Lobdell,"Israel Broussard, Suraj Sharma, Phi Vu",78,2019-02-13,Happy Death Day 2U,9000000,28051045,64179495,2019,55179495,6.131055
1541,tt8266310,2019,Blinded by the Light,Blinded by the Light,"Biography,Comedy,Drama",117.0,Gurinder Chadha,Sarfraz Manzoor,"Kulvinder Ghir, Aaron Phagura, Viveik Kalra",64,2019-08-14,Blinded by the Light,15000000,0,0,2019,-15000000,-1.000000
1544,tt8632862,2018,Fahrenheit 11/9,Fahrenheit 11/9,Documentary,128.0,Michael Moore,None,None,29,2018-09-21,Fahrenheit 11/9,5000000,6352306,6653715,2018,1653715,0.330743
1545,tt8852552,2010,Icarus,Icarus,Thriller,78.0,Maarten Diederen,Emile Beulen,None,91,2010-02-09,Icarus,6000000,0,0,2010,-6000000,-1.000000
